In [67]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

In [90]:
name=['LG 에어로타워']
category=['별점']

#LG 에어로타워 후기
ns_address="https://search.shopping.naver.com/catalog/30128278618?cat_id=50002543&frm=NVSCPRO&query=%EC%97%90%EC%96%B4%EB%A1%9C%ED%83%80%EC%9B%8C&NaPm=ct%3Dl0ksn0vc%7Cci%3D5bbd25c0299ce5dbcb72ff2b1d41488ebd6d52ce%7Ctr%3Dsls%7Csn%3D95694%7Chk%3D87194ce8ced4cb2b52968022b8eb9db67602d12e"
#xpath
#shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul"
shoppingmall_review="/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul"

In [92]:
header = {'User-Agent': 'Mozila/2.0'}
d = webdriver.Chrome() # webdriver = chrome
d.implicitly_wait(3)
d.get(ns_address)
req = requests.get(ns_address, headers = header)
html = req.text 
soup = BeautifulSoup(html, "html.parser")
sleep(2)

#쇼핑몰 리뷰 보기
d.find_element(By.XPATH, shoppingmall_review).click()
sleep(2)

element=d.find_element(By.XPATH, shoppingmall_review)
d.execute_script("arguments[0].click();", element)
sleep(2)

In [94]:
def add_dataframe(name,category,reviews,stars,cnt):  #데이터 프레임에 저장
    #데이터 프레임생성
    df1=pd.DataFrame(columns=['type','category','review','star'])
    n=1
    if (cnt>0):
        for i in range(0,cnt-1):
            df1.loc[n]=[name,category,reviews[i],stars[i]] #해당 행에 저장
            i+=1
            n+=1
    else:
        df1.loc[n]=[name,category,'null','null']
        n+=1    
    return df1

In [96]:
# 리뷰 가져오기 + 구매처 + 날짜 추가
d.find_element(By.XPATH,shoppingmall_review).click() #스크롤 건드리면 안됨
name_=name[0]
#category_=category[0]
shops=[]
dates=[]
reviews=[]
stars=[]
cnt=1   #리뷰index
page=1

In [98]:
while True:
    j=1
    print ("페이지", page ,"\n") 
    sleep(2)
    while True: #한페이지에 20개의 리뷰, 마지막 리뷰에서 error발생
        try:
            star=d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[1]').text
            stars.append(star)
            review=d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]').text
            reviews.append(review)
            shop = d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[2]').text
            shops.append(shop)
            date = d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[1]/span[4]').text
            dates.append(date)
            
            if j%2==0: #화면에 2개씩 보이도록 스크롤
                ELEMENT = d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li['+str(j)+']/div[2]/div[1]')
                d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)       
            j+=1
            print(cnt, shop, date, review ,star, "\n")
            cnt+=1 
        except: break
            
    sleep(2)
    
    if page<11:#page10
        try: #리뷰의 마지막 페이지에서 error발생
            page +=1
            next_page=d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page)+']').click() 
            
        except: break #리뷰의 마지막 페이지에서 process 종료
     
    else:
        try: #page11부터
            page+=1
            if page%10==0: next_page=d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a[11]').click()
            else : next_page=d.find_element(By.XPATH, '/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a['+str(page%10+2)+']').click()
                
        except: break
            

df4=add_dataframe(name_,shops,dates,reviews,stars,cnt)
#save()

페이지 1 

1 LG전자인증점 경남 22.01.22. 1/28 배송 예정이었는데 일찍 와서 좋았어요! 기사님께서 시간 정확히 맞춰서 빠른 설치해주시고 가셨습니다. 복도 가장 끝집이라 아침에 일어나면 너무너무 추워서 ㅠㅠ 온풍기를 들일
1/28 배송 예정이었는데 일찍 와서 좋았어요! 기사님께서 시간 정확히 맞춰서 빠른 설치해주시고 가셨습니다.
복도 가장 끝집이라 아침에 일어나면 너무너무 추워서 ㅠㅠ 온풍기를 들일 생각을 하고 있었는데 작년 12월에 에어로타워 광고를 보고 이거다 싶었죠!! 가전은 무조건 엘지잖아요? +_+ 디자인에 기능에 혹해서 결제해버렸어요.
엘지 띵큐 앱이랑 연동하면 전원 켜는 시간도 설정이 되니 아침 일어나는 시간에 맞춰서 거실을 따끈하게 데워놓을 생각에 너무 신나요!
집에서 한 잔하다가 동영상 리뷰 올리면 5000포인트 준다는 얘기 듣고 동영상 찍으려고 갑자기 일어나는데 ㅋㅋ 술을 쏟고 영상 찍었더니 집안 냄새가 나쁨으로 찍혔어요. 환기 시킨다고 창문 열었더니 미세먼지 안좋은 것도 바로바로 감지하네요!
여러분 ,, 고민은 진심 배송만 늦출 뿐입니다. 얼른 집에 들이십셔!! 평점5 

2 LG전자인증점 경남 22.01.22. 온풍기능 아주 좋습니다. 겨울에
온풍기능 아주 좋습니다. 겨울에도 쾌적하게 실내공기를 정화하면서 온도 조절까지 되니 좋네요 ~ 다만 앱으로 새부적인 기능들을 컨트롤 할 수 없는게 아쉽네요~ 예를 들어 몇시부터 몇시까지는 온풍으로 사용하다가 몇시부터는 공기를 위로 보낸다던지~ 아마 앱이 업데이트되면 지원되겠죠 디자인은 깔끔하니 예쁩니다. 배송도 다른 쇼핑몰에 비해 굉장히 빨리 온 편이라 만족합니다. 평점5 

3 LG전자인증점 경남 22.01.22. 신상품이라 먼저 쓸수있도록 빠른배송 감사합니다 기사님
신상품이라 먼저 쓸수있도록 빠른배송 감사합니다
기사님도 친절하게 시간맞춰서 오셨어요.
히터기능이 좋아요 겨울에 난방비도 절약이 될 것 같습니다
심지어 온도를 높게 설정하면 더워요;;
25도 정도가 적당하고 졸음 살살올정도

TypeError: add_dataframe() takes 5 positional arguments but 6 were given

In [102]:
df4.to_excel('NAVER_AIR_0828.xlsx')